## Introduction to segmentation through deep learning

## Tutorial 2. Lung segmentation in Chest X-Rays 

This lesson applies a [U-Net for Semantic Segmentation](https://arxiv.org/abs/1505.04597) of the lung fields on chest x-rays.

### Download the dataset

User can download many publicly available datasets like [Montgomery County dataset](https://lhncbc.nlm.nih.gov/LHC-publications/pubs/TuberculosisChestXrayImageDataSets.html), [JSRT dataset](http://db.jsrt.or.jp/eng.php) and [covid 19 dataset](https://github.com/v7labs/covid-19-xray-dataset) etc.  and train the lung segmentation model using those datasets. However due to time constraints, we will only train on montgomery dataset in this exercise.

This tutorial and use of the public datasets are for education purpose only.


**Clink the badge above to launch this notebook on Google Colab.**
To use GPU, go to Runtime -> Change runtime type -> and turn on GPU.

In [ ]:
!pip install pandas
!pip install tensorflow
!pip install SimpleITK
!pip install scikit-learn
!pip install opencv-python
!pip install pydicom
!pip install segmentation_models
!pip install scikit-image                                                                                                                                                                                                                                                                          

### Import all the libraries

In [ ]:
import glob
import os
import cv2
import h5py
import random
import numpy as np
import SimpleITK as sitk
import tensorflow as tf
from skimage import transform, io, img_as_float, exposure
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Convolution2D, MaxPooling2D, UpSampling2D, concatenate

### Mount your Google drive and allow the colab to access your Google drive 

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

root_dir = '/content/drive/MyDrive/niehs_tutorials'

### Combine left and right lung masks from Montgomery County dataset and prepare load data function.

In [ ]:
def combine_right_left_lung_masks(data_dir_path):
    for filename in glob.glob(os.path.join(data_dir_path,'CXR_png','*.png')):
        print('Combined right and left lung masks for '+filename) 
        left = io.imread(os.path.join(data_dir_path,'ManualMask','leftMask', os.path.basename(filename)))
        right = io.imread(os.path.join(data_dir_path,'ManualMask','rightMask', os.path.basename(filename)))
        io.imsave(os.path.join(data_dir_path,'combined_masks', os.path.basename(filename)), left+right)


def load_data(data_dir_path,resize_shape):
    #combine_right_left_lung_masks(data_dir_path)
    cxr_filenames = glob.glob(os.path.join(data_dir_path,"CXR_png",'*.png'))  # CXR paths
    masks_filenames = glob.glob(os.path.join(data_dir_path,"combined_masks",'*.png'))  # Masks paths

    X = []
    Y = []
    for file in  cxr_filenames:
          cxr = sitk.ReadImage(file)
          mask = sitk.ReadImage(os.path.join(data_dir_path,"combined_masks",os.path.basename(file)))
          new_spacing = [sz*spc/nsz for nsz,sz,spc in zip(resize_shape, cxr.GetSize(), cxr.GetSpacing())]

          resampled_org = sitk.Resample(cxr, resize_shape, sitk.Transform(), sitk.sitkLinear,
                                            cxr.GetOrigin(), new_spacing, cxr.GetDirection(),
                                            0, sitk.sitkFloat32)
          resampled_seg = sitk.Resample(mask, resize_shape, sitk.Transform(), sitk.sitkNearestNeighbor,
                                            mask.GetOrigin(), new_spacing, mask.GetDirection(),
                                            0, sitk.sitkInt32)
          img = sitk.GetArrayFromImage(resampled_org)
          seg = sitk.GetArrayFromImage(resampled_seg)
          #Standardize image between [0,1]
          img = (img - img.min()) / (img.max() - img.min())
          img = exposure.equalize_adapthist(img)
          img = np.expand_dims(img, axis=-1)
          seg = seg/255
          X.append(img)
          Y.append(seg)

    X = np.asarray(X)
    Y = np.asarray(Y)

    return X,Y

### Load data 

In [ ]:
data_dir_path = os.path.join(root_dir,'MontgomerySet')
X,Y = load_data(data_dir_path,resize_shape=(256,256))

### Visualize data when right and left lung amsks are combined.

In [ ]:
num_samples = 5

plt.rcParams["figure.figsize"]=10,10
_,subfigs = plt.subplots(num_samples, 2)
for sample in range(num_samples):
      arr = X[sample,:,:,0]
      ground_truth = Y[sample,:,:]
      subfigs[sample,0].imshow(arr, cmap = 'gray')
      subfigs[sample,0].set_title('Sample Image #'+str(sample))
      subfigs[sample,0].axis('off')
      subfigs[sample,1].imshow(ground_truth, cmap = 'gray')
      subfigs[sample,1].set_title('Ground Truth #'+str(sample))
      subfigs[sample,1].axis('off')
plt.show()

### Divide train/val/test sets for model development

In [ ]:
# train-val-test split 
random.Random(4).shuffle(X)
random.Random(4).shuffle(Y)
split_1 = int(0.8 * len(X))
split_2 = int(0.9 * len(X))
X_train,Y_train = X[:split_1],Y[:split_1]
X_val,Y_val = X[split_1:split_2],Y[split_1:split_2]
X_test,Y_test = X[split_2:],Y[split_2:]

### Build the model

In [ ]:
def UNet(inp_shape, k_size=3):
    merge_axis = -1 # Feature maps are concatenated along last axis (for tf backend)
    data = Input(shape=inp_shape)
    conv1 = Convolution2D(filters=32, kernel_size=k_size, padding='same', activation='relu')(data)
    conv1 = Convolution2D(filters=32, kernel_size=k_size, padding='same', activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(pool1)
    conv2 = Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(pool2)
    conv3 = Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(pool3)
    conv4 = Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(pool4)

    up1 = UpSampling2D(size=(2, 2))(conv5)
    conv6 = Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(up1)
    conv6 = Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(conv6)
    merged1 = concatenate([conv4, conv6], axis=merge_axis)
    conv6 = Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(merged1)

    up2 = UpSampling2D(size=(2, 2))(conv6)
    conv7 = Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(up2)
    conv7 = Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(conv7)
    merged2 = concatenate([conv3, conv7], axis=merge_axis)
    conv7 = Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(merged2)

    up3 = UpSampling2D(size=(2, 2))(conv7)
    conv8 = Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(up3)
    conv8 = Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(conv8)
    merged3 = concatenate([conv2, conv8], axis=merge_axis)
    conv8 = Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(merged3)

    up4 = UpSampling2D(size=(2, 2))(conv8)
    conv9 = Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(up4)
    conv9 = Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(conv9)
    merged4 = concatenate([conv1, conv9], axis=merge_axis)
    conv9 = Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(merged4)

    conv10 = Convolution2D(filters=1, kernel_size=k_size, padding='same', activation='sigmoid')(conv9)

    output = conv10
    model = Model(data, output)
    return model

In [ ]:
epochs = 100
batch_size = 8
learning_rate = 0.0001 

model = UNet((256,256,1), k_size=3)

opt = optimizers.Adam(learning_rate=learning_rate)  # ,momentum=.8,nesterov=True) #.00001

model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['accuracy'],
                  )

Train the model

In [ ]:
# Callbacks to stop early and save model periodically
model_output_filename = os.path.join(root_dir,'best_lung_segment_model.h5')
earlystopper = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)
modelSaver = ModelCheckpoint(model_output_filename, monitor='val_loss', verbose=0,
                              save_best_only=True,
                              save_weights_only=False, mode='auto', save_freq='epoch')

model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val),
              callbacks=[earlystopper, modelSaver])

### Test the model

In [ ]:
## Define metrics

# Intersection Over Union
def IoU(y_true, y_pred):
    
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.logical_and(y_true_f, y_pred_f).sum()
    union = np.logical_or(y_true_f, y_pred_f).sum()
    return (intersection + 1) * 1. / (union + 1)

# Dice coefficient
def Dice(y_true, y_pred):

    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.logical_and(y_true_f, y_pred_f).sum()
    return (2. * intersection + 1.) / (y_true.sum() + y_pred.sum() + 1.)

#Misclassification rate
def misclassification_rate(y_true,y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    fn = tf.keras.metrics.FalseNegatives()
    fn.update_state(y_true_f,y_pred_f)
    fn = fn.result().numpy()
    fp = tf.keras.metrics.FalsePositives()
    fp.update_state(y_true_f,y_pred_f)
    fp = fp.result().numpy()
    tp = tf.keras.metrics.TruePositives()
    tp.update_state(y_true_f,y_pred_f)
    tp =tp.result().numpy()
    tn = tf.keras.metrics.TrueNegatives()
    tn.update_state(y_true_f,y_pred_f) 
    tn = tn.result().numpy()
    misclassification_rate = (fp+fn)/(fp+fn+tp+tn)  
    return misclassification_rate
    


In [ ]:
model_output_filename = os.path.join(root_dir,'best_lung_segment_model.h5')
model = tf.keras.models.load_model(model_output_filename)
preds = model.predict(X_test,batch_size=batch_size)


# Binarize masks with the threshold
threshold=0.5
preds = preds > threshold

ious = np.zeros(len(X_test))
dices= np.zeros(len(X_test))
misclassify_rates = np.zeros(len(X_test))

for i in range(len(X_test)):
     ious[i] = IoU(preds[i],Y_test[i])
     dices[i] = Dice(preds[i],Y_test[i])
     misclassify_rates[i] = misclassification_rate(preds[i],Y_test[i])

print("Mean Dice:"+str(dices.mean()))
print("Mean IoU:"+str(ious.mean()))
print("Misclassification rate:"+str(misclassify_rates.mean()))

### Visualize the results

In [ ]:
num_samples = 5

plt.rcParams["figure.figsize"]=10,10
_,subfigs = plt.subplots(num_samples, 3)
for sample in range(num_samples):
      arr = X_test[sample,:,:,0]
      ground_truth = Y_test[sample,:,:]
      prediction = preds[sample,:,:,0].astype(int)
      subfigs[sample,0].imshow(arr, cmap = 'gray')
      subfigs[sample,0].set_title('Test Image #'+str(sample))
      subfigs[sample,0].axis('off')
      subfigs[sample,1].imshow(ground_truth, cmap = 'gray')
      subfigs[sample,1].set_title('Ground Truth #'+str(sample))
      subfigs[sample,1].axis('off')
      subfigs[sample,2].imshow(prediction, cmap = 'gray')
      subfigs[sample,2].set_title('Prediction #'+str(sample))
      subfigs[sample,2].axis('off')

plt.show()

#### Now perform morphological operations to see if you can improve the overall performance

In [ ]:
def dilate_prediction(prediction,radius=6):
    sample = sitk.GetImageFromArray(prediction.astype(int))
    dilateFilter = sitk.DilateObjectMorphologyImageFilter()
    dilateFilter.SetKernelRadius(radius)
    out = dilateFilter.Execute(sample)
    out_arr = sitk.GetArrayFromImage(out)
    return out_arr

def erode_prediction(prediction,radius=6):
    sample = sitk.GetImageFromArray(prediction.astype(int))
    erodeFilter = sitk.BinaryErodeImageFilter()
    erodeFilter.SetKernelRadius(radius)
    out = erodeFilter.Execute(sample)
    out_arr = sitk.GetArrayFromImage(out)
    return out_arr

def close_prediction(prediction,radius=6):
    sample = sitk.GetImageFromArray(prediction.astype(int))
    closingFilter = sitk.BinaryMorphologicalClosingImageFilter()
    closingFilter.SetKernelRadius(radius)
    closingFilter.SetForegroundValue(1)
    out = closingFilter.Execute(sample)
    out_arr = sitk.GetArrayFromImage(out)
    return out_arr

def open_prediction(prediction,radius=6):
    sample = sitk.GetImageFromArray(prediction.astype(int))
    openingFilter = sitk.BinaryMorphologicalOpeningImageFilter()
    openingFilter.SetKernelRadius(radius)
    openingFilter.SetForegroundValue(1)
    out = openingFilter.Execute(sample)
    out_arr = sitk.GetArrayFromImage(out)
    return out_arr


### Check performance of the model predictions when each of the morphological operations are performed. User canfeel free to change the kernel radius for each of the morphological operation fucntion and see how the performance varies

In [ ]:
ious = np.zeros(len(X_test))
dices= np.zeros(len(X_test))
misclassify_rates = np.zeros(len(X_test))
for i in range(len(X_test)):
     ious[i] = IoU(dilate_prediction(preds[i],radius=1),Y_test[i])
     dices[i] = Dice(dilate_prediction(preds[i],radius=1),Y_test[i])
     misclassify_rates[i] = misclassification_rate(dilate_prediction(preds[i],radius=1),Y_test[i])

print("Mean Dice when dilation filter is applied:"+str(dices.mean()))
print("Mean IoU when dilation filter is applied::"+str(ious.mean()))
print("Misclassification rate:"+str(misclassify_rates.mean()))

ious = np.zeros(len(X_test))
dices= np.zeros(len(X_test))
misclassify_rates = np.zeros(len(X_test))
for i in range(len(X_test)):
     ious[i] = IoU(erode_prediction(preds[i],radius=1),Y_test[i])
     dices[i] = Dice(erode_prediction(preds[i],radius=1),Y_test[i])
     misclassify_rates[i] = misclassification_rate(erode_prediction(preds[i],radius=1),Y_test[i])

print("Mean Dice when erosion filter is applied:"+str(dices.mean()))
print("Mean IoU when erosion filter is applied::"+str(ious.mean()))
print("Misclassification rate:"+str(misclassify_rates.mean()))

ious = np.zeros(len(X_test))
dices= np.zeros(len(X_test))
misclassify_rates = np.zeros(len(X_test))
for i in range(len(X_test)):
     ious[i] = IoU(close_prediction(preds[i],radius=1),Y_test[i])
     dices[i] = Dice(close_prediction(preds[i],radius=1),Y_test[i])
     misclassify_rates[i] = misclassification_rate(close_prediction(preds[i],radius=1),Y_test[i])

print("Mean Dice when morpholical closing filter is applied:"+str(dices.mean()))
print("Mean IoU when morpholical closing filter is applied::"+str(ious.mean()))
print("Misclassification rate:"+str(misclassify_rates.mean()))

ious = np.zeros(len(X_test))
dices= np.zeros(len(X_test))
misclassify_rates = np.zeros(len(X_test))
for i in range(len(X_test)):
     ious[i] = IoU(open_prediction(preds[i],radius=1),Y_test[i])
     dices[i] = Dice(open_prediction(preds[i],radius=1),Y_test[i])
     misclassify_rates[i] = misclassification_rate(open_prediction(preds[i],radius=1),Y_test[i])

print("Mean Dice when morphological opening filter is applied:"+str(dices.mean()))
print("Mean IoU when morphological opening filter is applied::"+str(ious.mean()))
print("Misclassification rate:"+str(misclassify_rates.mean()))

### From the above snippet, determine which postprocessing filter is giving better performance with the ground truths than the original model predictions.                                                           